In [57]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [77]:
df_jan = pd.read_parquet('C:/Users/Fozan/Downloads/mlops-zoomcamp-hw/data/yellow_tripdata_2023-01.parquet')

In [79]:
df_jan

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.00,0.97,1.00,N,161,141,2,9.30,1.00,0.50,0.00,0.00,1.00,14.30,2.50,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.00,1.10,1.00,N,43,237,1,7.90,1.00,0.50,4.00,0.00,1.00,16.90,2.50,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.00,2.51,1.00,N,48,238,1,14.90,1.00,0.50,15.00,0.00,1.00,34.90,2.50,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.00,1.90,1.00,N,138,7,1,12.10,7.25,0.50,0.00,0.00,1.00,20.85,0.00,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.00,1.43,1.00,N,107,79,1,11.40,1.00,0.50,3.28,0.00,1.00,19.68,2.50,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.50,3.96,0.00,1.00,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.50,2.64,0.00,1.00,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.50,5.32,0.00,1.00,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.50,4.43,0.00,1.00,26.58,NaN,NaN


In [81]:
df_jan.shape

(3066766, 19)

In [82]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [83]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [84]:
df_jan['duration'].std()

42.594351241920904

In [85]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]


In [86]:
df_jan.shape

(3009173, 20)

In [65]:
df.duration.describe(percentiles=[0.90,0.92,0.95,0.98])

count   3066766.00
mean         15.67
std          42.59
min         -29.20
50%          11.52
90%          27.94
92%          30.55
95%          36.47
98%          48.73
max       10029.18
Name: duration, dtype: float64

In [87]:
left_data = 3009173 / 3066766 * 100
left_data

98.1220282212598

In [69]:
categorical = ['PULocationID', 'DOLocationID']

In [89]:
df_jan[categorical] = df_jan[categorical].astype(str)

C:\Users\Fozan\AppData\Local\Temp\ipykernel_11632\1583698975.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan[categorical] = df_jan[categorical].astype(str)


In [90]:
df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [91]:
train_dicts = df_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_jan[target].values

X_train.shape

(3009173, 515)

In [92]:
num_columns = X_train.shape[1]
num_columns

515

In [93]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649261957307838

In [94]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']

df_feb[categorical] = df_feb[categorical].astype(str)

train_dicts = df_feb[categorical].to_dict(orient='records')

X_train = dv.transform(train_dicts)

target = 'duration'
y_train = df_feb[target].values

NameError: name 'df_feb' is not defined

In [95]:
y_pred_feb = lr.predict(X_train)

mean_squared_error(y_train, y_pred_feb, squared=False)

7.649261957307838

In [ ]:
# Save your model and dv as "model_dv.bin"

with open("model_dv.bin", "wb") as f:
    pickle.dump((lr, dv), f)